In [224]:
import requests
import pandas as pd
import json
from pyspark.sql import SparkSession

In [77]:
%run ./AzureKeyVault.ipynb

In [226]:
spark = SparkSession.builder.appName("TrafficForecasting").getOrCreate()

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/opt/anaconda3/lib/python3.12/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [184]:
class USCensusAPI:
    def __init__(self, dataset, cols, location):
        self.host = 'https://api.census.gov/data'
        self.dataset = dataset
        self.cols = variables
        self.location = location
        self.user_key = KeyVault().get_secret("CensusAPIKey")

    def get_query_url(self):
        return f"{self.host}/{self.dataset}?get={self.cols}&for={self.location}&key={self.user_key}"

    def query_results(self):
        return requests.get(self.get_query_url()).text

    def query_output(self):
        print(self.query_results())

    def query_df(self):
        data = json.loads(self.query_results())
        return pd.DataFrame(
            data[1:],
            columns=data[0]
        )

In [222]:
class AustinDataAPI:
    def __init__(self, filter=None, limit=None):
        self.url = 'https://data.austintexas.gov/resource/dx9v-zd7x.json'
        self.app_token = KeyVault().get_secret("AustinDataToken")
        self.filter = filter
        self.limit = limit
        self.page = page

    def query_results(self, offset=None):
        params = {"$$app_token": self.app_token}
        
        if self.filter:
            params.update({"$where": self.filter})
        if self.limit:
            params.update({"limit": self.limit})
        if self.page:
            params.update({"page": self.page})
        if offset:
            params.update({"$offset": offset))
            
        response = requests.get(self.url, params=params)
        response.raise_for_status()
        return response.json()

    def query_df(self):
        return pd.DataFrame.from_records(self.query_results().json())

    def get_total_records(self):
        params = { "$select": "count(*)" }        
        response = requests.get(self.url, params=params)
        response.raise_for_status()
        return response.json()

    def process_batches(self, batch_size=1000):
        offsets = range(0, self.get_total_records(), batch_size)
    
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(query_results, offsets))

        return [item for sublist in results for item in sublist]

    def get_all_data(self, batch_size=1000):
        return pd.DataFrame(self.process_batches())